In [1]:
import pymupdf
import re

In [62]:
import fitz  # PyMuPDF
import re

def extract_questions_and_choices_from_pdf(pdf_path):
    # Ouvrir le fichier PDF
    document = fitz.open(pdf_path)
    text = ""

    # Extraire le texte de chaque page
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()

    # Utilisation d'une expression régulière pour trouver les blocs de questions
    question_blocks = re.split(r'(?=Question \d+\s)', text)

    questions = []

    for block in question_blocks:
        if block.strip():  # Vérifie si le bloc n'est pas vide
            # Extraire le numéro de la question
            question_match = re.search(r'Question (\d+)', block)
            if question_match:
                question_number = question_match.group(1)

                # Extraire le texte de la question
                question_text_match = re.search(r'Question \d+.*?(?=\n[A-Z]\.\s|\n\d+\.\s|Answer)', block, re.DOTALL)
                question_text = question_text_match.group(0).strip() if question_text_match else ""

                # Extraire les choix possibles
                choices = []
                choices_match = re.findall(r'([A-Z]\.\s.+?)(?=\n[A-Z]\.\s|\n\d+\.\s|\nAnswer|\Z)', block, re.DOTALL)
                for choice in choices_match:
                    choices.append(choice.strip())

                # Extraire la réponse correcte
                answer_match = re.search(r'Answer\s*\n*(.+)', block, re.DOTALL)
                answer = answer_match.group(1).strip() if answer_match else ""

                if question_text == '':
                    continue
                # Ajouter la question et les choix à la liste
                questions.append({
                    "question_number": question_number,
                    "question_text": question_text[len(question_match.group(0)):].strip('\n'),
                    "answer": answer
                })

    return questions

# Exemple d'utilisation
pdf_path = "../../resources/unprocess/Questions Sup OEB.docx.pdf"
questions = extract_questions_and_choices_from_pdf(pdf_path)

import json
# dump json
with open('EOB.json', 'w') as f:
    json.dump(questions, f)

print(questions[8])
# for question in questions:

#     print(f"Question {question['number']}: {question['text']}")
#     print('------------------\n\n')
    
    # for choice in question['choices']:
    #     print(f" - {choice}")
    # print(f"Answer: {question['answer']}\n")


{'question_number': '9', 'question_text': ' \n \nMost fees for European patent applications cannot be paid before their due date. One \nexception is renewal fees, which can always be paid up to: \nA.\u200b\nThree months before the due date. \nB.\u200b\nSix months before the due date. \nC.\u200b\nOne year before the due date.', 'answer': 'The correct answer is A.  \nRenewal fees for European patent applications may be validly paid up to three months before \nthe due date. For the  renewal fee for the third year, answer B is also correct. This special \nrule for the 3rd renewal fee was made to allow for the payment of this fee on entry into the \nregional phase (Rule 51(1) EPC, implementing Article 86(1) EPC).'}


In [46]:
import re

def extract_choices_from_question_block(question_block):
    # Utilisation de l'expression régulière pour trouver les choix possibles
    regex_pattern = r'(?m)^([A-Z0-9]{1}\..*)'
    choices_pattern = re.compile(regex_pattern)
    choices = choices_pattern.findall(question_block)

    
    # split text by choices
    choices = re.split(regex_pattern, question_block)
    context = choices.pop(0)
    # print(context)
    # print('====================')
    
    assert len(choices) % 2 == 0, "The number of choices is not even"
    
    # join each 2 choices
    choices_final = [choices[i] + choices[i+1] for i in range(0, len(choices) - 1, 2)]

    for choice in choices_final:
        print(choice)
        print('-------------------')


    # Nettoyage des choix extraits
    choices = [choice.strip() for choice in choices]

    return choices

# Exemple d'utilisation
for question in questions[:4]:
    
    print(question['number'])
    choices = extract_choices_from_question_block(question['text'])
    # print(question['text'])
    # for i, choice in enumerate(choices, start=1):
    #     print(f"Choice {i}: {choice}")



1
A.​
The applicant is entitled to a reduction of 20% in the examination fee under Rule 6(3) 
EPC and Article 14(1) RFees. 
  

-------------------
B.​
The applicant is entitled to a reduction in the examination fee under Rule 6(3)EPC 
and Article 14(1) RFees, but the reduction is not 20%.
-------------------
2
A.​
True 

-------------------
B.​
False
-------------------
3
4
1.​ Obtain the consent of Mr Y to rectification of the inventor before filing a request at the 
International Bureau with details of the correct inventor.​
  

-------------------
2.​ File a request for rectification at the International Bureau with details of the correct 
inventor.
-------------------
